In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import torch
from torch.utils.data import DataLoader, ConcatDataset, SubsetRandomSampler
from sklearn.model_selection import StratifiedKFold, KFold
import numpy as np

from dataset import ImageTableDataset
from model import MultimodalModel
from train import train

In [17]:
# ----- Dataset -----
print("Loading datasets...")

batch_size = 2

train_dataset = ImageTableDataset("train_morbidity")
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = ImageTableDataset("val_morbidity")
# val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

dataset = ConcatDataset([train_dataset, val_dataset])
labels = [dataset[i]['label'] for i in range(len(dataset))]

k = 2
splits = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

Loading datasets...


In [18]:
len(dataset)

1222

In [19]:
labels = [dataset[i]['label'] for i in range(len(dataset))]

In [21]:
# labels

In [8]:
splits.split(dataset, labels)

<generator object _BaseKFold.split at 0x156534c10>

In [22]:
# ----- Training -----
print("Start training...")
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
config = {
    "learning_rate": 0.001,
    "max_epochs": 50,
    "print_every_n": 10,
    "validate_every_n": 10,
    "experiment_name": "baseline"
}

Start training...


In [7]:
# ----- Model -----
print("Creating model...")
dd

Creating model...


2022-12-19 14:41:23.800525: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [27]:
 model = MultimodalModel()

In [44]:
results = {"val_acc": [], "val_auc": []}
for fold, (train_idx, val_idx) in enumerate(splits.split(dataset, labels)):
    print('Fold {}'.format(fold + 1))
    
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    train_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    val_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)
    
    model = MultimodalModel()

    history = train(model, train_dataloader, val_dataloader, device, config)
    results["val_acc"].append(history["val_acc"])
    results["val_auc"].append(history["val_auc"])
#     results["val_sn"].append(history["val_sn"])
#     results["val_sp"].append(history["val_sp"])
#     results["val_ppv"].append(history["val_ppv"])
#     results["val_npv"].append(history["val_npv"])
#     results["val_mcc"].append(history["val_mcc"])

print("Validation accuracy:", torch.mean(torch.Tensor(results["val_acc"])).item())
print("Validation AUC:", torch.mean(torch.Tensor(results["val_auc"])).item())
# print("Validation SN (Recall):", torch.mean(results["val_sn"]).item())
# print("Validation SP (Specificity):", torch.mean(results["val_sp"]).item())
# print("Validation PPV:", torch.mean(results["val_ppv"]).item())
# print("Validation NPV:", torch.mean(results["val_npv"]).item())
# print("Validation MCC:", torch.mean(results["val_mcc"]).item())

Fold 1
[000/00009] train_loss: 1.091
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
pr

predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1, 1])
predicted label tensor([1])


ValueError: multi_class must be in ('ovo', 'ovr')

In [35]:
import torch.nn as nn

In [36]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)

In [39]:
input.shape

torch.Size([3, 5])

In [40]:
target.shape

torch.Size([3])

In [ ]:
output.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
output.backward()